In [ ]:
!pip install -U sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
--- Logging error ---
Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.8/site-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/home/ubuntu/.local/lib/python3.8/site-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/home/ubuntu/.local/lib/python3.8/site-packages/pip/_vendor/rich/console.py", line 1305, in render
    for render_output in iter_render:
  File "/home/ubuntu/.local/lib/python3.8/site-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/home/ubuntu/.local/lib/python3.8/site-packages/pip/_vendor/rich/segment.py", line 249, in split_lines
    for segment in segments:
  File "/home/ubuntu/.local/lib/python3.8/site-packages/pip/_vendor/rich/console.py", line 1283, in

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from transformers import GPT2Tokenizer, GPT2Model, GPT2LMHeadModel
import torch
from transformers import RobertaConfig, RobertaModel, RobertaTokenizer, RobertaModel
import math
# import Model_Import
import Model_Import_6 ## eating mem ## changed version commented out above
from torch import optim
import random

In [ ]:
from transformers import RobertaConfig, RobertaModel, RobertaTokenizer, RobertaModel

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
R_tokenizer = RobertaTokenizer.from_pretrained("roberta-large")
Roberta_model = RobertaModel.from_pretrained("roberta-large").to(device)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:



text = """And, after boasting this way of my tolerance, I come to the admission
that it has a limit. Conduct may be founded on the hard rock or the
wet marshes, but after a certain point I don’t care what it’s founded
on. When I came back from the East last autumn I felt that I wanted
the world to be in uniform and at a sort of moral attention forever; I
wanted no more riotous excursions with privileged glimpses into the
human heart. Only Gatsby, the man who gives his name to this book, was
exempt from my reaction—Gatsby, who represented everything for which I
have an unaffected scorn. If personality is an unbroken series of
successful gestures, then there was something gorgeous about him, some
heightened sensitivity to the promises of life, as if he were related
to one of those intricate machines that register earthquakes ten
thousand miles away. This responsiveness had nothing to do with that
flabby impressionability which is dignified under the name of the
“creative temperament”—it was an extraordinary gift for hope, a
romantic readiness such as I have never found in any other person and
which it is not likely I shall ever find again. No—Gatsby turned out
all right at the end; it is what preyed on Gatsby, what foul dust
floated in the wake of his dreams that temporarily closed out my
interest in the abortive sorrows and short-winded elations of men.
"""
text = text.replace('\n', ' ')
sent_text = nltk.sent_tokenize(text)
print(sent_text)


['And, after boasting this way of my tolerance, I come to the admission that it has a limit.', 'Conduct may be founded on the hard rock or the wet marshes, but after a certain point I don’t care what it’s founded on.', 'When I came back from the East last autumn I felt that I wanted the world to be in uniform and at a sort of moral attention forever; I wanted no more riotous excursions with privileged glimpses into the human heart.', 'Only Gatsby, the man who gives his name to this book, was exempt from my reaction—Gatsby, who represented everything for which I have an unaffected scorn.', 'If personality is an unbroken series of successful gestures, then there was something gorgeous about him, some heightened sensitivity to the promises of life, as if he were related to one of those intricate machines that register earthquakes ten thousand miles away.', 'This responsiveness had nothing to do with that flabby impressionability which is dignified under the name of the “creative temperame

In [ ]:
!pwd

/home/ubuntu


In [ ]:
smal_g_df = pd.read_csv('small_gutenberg_data.csv', header = None)
smal_g_df

,0,1,2,3,4,5
0,Title,Author,Link,ID,Bookshelf,Text
1,The Extermination of the American Bison,William T. Hornaday,http://www.gutenberg.org/ebooks/17748,17748,Animal,[Illustration: (Inscription) Mr. Theodore Roos...
2,Deadfalls and Snares,A. R. Harding,http://www.gutenberg.org/ebooks/34110,34110,Animal,DEADFALLS AND SNARES [Frontispiece: A GOOD DEA...
3,Artistic Anatomy of Animals,Édouard Cuyer,http://www.gutenberg.org/ebooks/38315,38315,Animal,+---------------------------------------------...
4,"Birds, Illustrated","Color Photography, Vol. 1, No. 1 Various",http://www.gutenberg.org/ebooks/30221,30221,Animal,FROM: THE PRESIDENT OF THE NATIONAL TEACHERS' ...
5,On Snake-Poison: Its Action and Its Antidote,A. Mueller,http://www.gutenberg.org/ebooks/32947,32947,Animal,[Illustration] ON SNAKE-POISON. ITS ACTION AND...
6,Fifty Years a Hunter and Trapper,E. N. Woodcock,http://www.gutenberg.org/ebooks/34063,34063,Animal,FIFTY YEARS A HUNTER AND TRAPPER [Frontispiece...
7,What Bird is That?,Frank M. Chapman,http://www.gutenberg.org/ebooks/31751,31751,Animal,Online Distributed Proofreading Team at https:...
8,Fox Trapping: A Book of Instruction Telling Ho...,NaN,http://www.gutenberg.org/ebooks/34076,34076,Animal,FOX TRAPPING [Frontispiece: FALL CATCH] FOX TR...
9,A Guide for the Study of Animals,"Lucas, Shinn, Smallwood, and Whitney",http://www.gutenberg.org/ebooks/34984,34984,Animal,Proofreading Team at https://www.pgdp.net Tran...


In [1]:
with open('/home/ubuntu/Darwin_Gatsby/gatsby_raw.txt', 'r') as file:
    gatsby = file.read().replace('\n', ' ')

with open('/home/ubuntu/Darwin_Gatsby/origin_of_species_raw.txt', 'r') as file:
    origin = file.read().replace('\n', ' ')
    
    #Darwin_Gatsby/origin_of_species_raw.txt

In [6]:
gatsby_sen_list = nltk.sent_tokenize(gatsby)
origin_sen_list = nltk.sent_tokenize(origin)
print(len(gatsby_sen_list))
print(len(origin_sen_list))

2439
3981


In [7]:
print(gatsby_sen_list[0])

﻿                               I  In my younger and more vulnerable years my father gave me some advice that I’ve been turning over in my mind ever since.


In [ ]:
def make_context_embeddings_with_mean(model, tokenizer, book_sentence_list):
    book_embeds = []
    count = 0
    with torch.no_grad():
        for sen in book_sentence_list:
            if len(sen) == 0:
                print("empty sentence")
                continue
            R_tokenized = tokenizer(sen, return_tensors = 'pt', truncation=True).to(device)
            #print(R_tokenized.shape)
            R_embed = model(**R_tokenized).last_hidden_state.squeeze()
            # print(R_embed.shape)
            #print(R_embed.shape)
            del R_tokenized
            R_embed = torch.mean(R_embed, 0)
            R_embed_cpu = R_embed.to('cpu')
            del R_embed
            book_embeds.append(R_embed_cpu)
            if count%1000 == 0:
                # t = torch.cuda.get_device_properties(0).total_memory
                # r = torch.cuda.memory_reserved(0)
                # a = torch.cuda.memory_allocated(0)
                # f = r-a  # free inside reserved
                # print(t/1000000000)
                # print(r/1000000000)
                # print(a/1000000000)
                # print(f/1000000000)
                print(count)
            count += 1
        return book_embeds

In [ ]:
# R_gatsby_embeds = make_context_embeddings_with_mean(Roberta_model, R_tokenizer, gatsby_sen_list)

In [ ]:
# R_origin_embeds = make_context_embeddings_with_mean(Roberta_model, R_tokenizer, origin_sen_list)

In [ ]:
# print(R_gatsby_embeds[0].shape)
# print(len(R_gatsby_embeds))
# # print(len(R_origin_embeds))

In [ ]:
# gpt_tokenizer = GPT2Tokenizer.from_pretrained("gpt2-xl")
# gpt_head_model = GPT2LMHeadModel.from_pretrained('gpt2-xl').to(device)
# gpt_head_transformer = gpt_head_model.transformer

In [ ]:
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
print(t/1000000000)
print(r/1000000000)
print(a/1000000000)
print(f/1000000000)

42.314694656
14.294188032
14.265847808
0.028340224


In [ ]:
def make_logits(model, tokenizer, book_sentence_list):
    logits = []
    token_ids = []
    count = 0
    with torch.no_grad():
        for sen in book_sentence_list:
            if len(sen) == 0:
                print("empty sentence")
                continue
            gpt_tokenized = tokenizer(sen, return_tensors = 'pt', truncation=True).to(device)
            gpt_embed = model(**gpt_tokenized).last_hidden_state.squeeze()
            gpt_tokenized_cpu = gpt_tokenized.to('cpu')
            del gpt_tokenized
            token_ids.append(gpt_tokenized_cpu) ## on the cpu!!!!
            gpt_embed_cpu = gpt_embed.to('cpu')
            del gpt_embed
            logits.append(gpt_embed_cpu)
            if count%1000 == 0:
                # t = torch.cuda.get_device_properties(0).total_memory
                # r = torch.cuda.memory_reserved(0)
                # a = torch.cuda.memory_allocated(0)
                # f = r-a  # free inside reserved
                # print(t/1000000000)
                # print(r/1000000000)
                # print(a/1000000000)
                # print(f/1000000000)
                print(count)
            count += 1
            
        return logits, token_ids

In [ ]:
# gatsby_logits, gatsby_token_ids = make_logits(Model_Import.head_transformer, Model_Import.tokenizer, gatsby_sen_list)

In [ ]:
# origin_logits, origin_token_ids = make_logits(Model_Import.head_transformer, Model_Import.tokenizer, origin_sen_list)

In [ ]:
# print(gatsby_logits[0].shape)
# print(len(gatsby_logits))
# # print(len(origin_logits))
# print(len(gatsby_token_ids))

In [ ]:
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
print(t/1000000000)
print(r/1000000000)
print(a/1000000000)
print(f/1000000000)

42.314694656
14.294188032
14.265847808
0.028340224


In [ ]:
# gatsby_token_ids[0]['input_ids']

In [ ]:
def train_one_style(model, optimizer, context_embeds_list, logits_list, token_ids_list, epochs, num_samples = 100):
    CELoss = nn.CrossEntropyLoss()
    total_count = 0
    for epoch in range(epochs):
        
        to_shuffle = list(zip(logits_list, token_ids_list))

        random.shuffle(to_shuffle)

        logits_list, token_ids_list = zip(*to_shuffle)

        model.train()
        ag_loss_epoch = 0
        epoch_count = 0
        for example in range(len(logits_list)):
            random_context_samples = random.sample(context_embeds_list, num_samples) # could use another context to see what happens
            stacked_context_sample = torch.stack(random_context_samples, dim = 0)
            # print(stacked_context_sample.shape)
            optimizer.zero_grad()
            network_output = model(stacked_context_sample.to(device), logits_list[example].to(device))
            if token_ids_list[example]['input_ids'].shape[1] == 1:
                print("ONE text id")
                continue
            shifted_network_output = network_output[..., :-1, :].contiguous()
            shifted_text_ids = token_ids_list[example]['input_ids'][..., 1:].contiguous().to(device)
            loss = CELoss(shifted_network_output.view(-1, shifted_network_output.size(-1)), shifted_text_ids.view(-1))
            ag_loss_epoch += loss
            epoch_count += 1
            total_count += 1
            loss.backward()
            optimizer.step()
        print(f"Epoch: {epoch}, Epoch Examples: {epoch_count}")
        print(f"TRAIN LOSS: {ag_loss_epoch / len(logits_list)}")
        # print(f"DEV LOSS: {full_dev_loss}")
        print("----------------------------------------")

In [ ]:
# R_gatsby_embeds[0].shape[0]

In [ ]:
# gatsby_alone_model = Model_Import.ProposedModel(R_gatsby_embeds[0].shape[0], gatsby_logits[0].shape[1], attention_dim = None)
# optimizer = optim.Adam(gatsby_alone_model.parameters(), lr=0.00001,  weight_decay=0.001)

In [ ]:
# train_one_style(gatsby_alone_model, optimizer, R_gatsby_embeds, gatsby_logits, gatsby_token_ids, 100)

In [ ]:
# origins_alone_model = Model_Import.ProposedModel(R_origin_embeds[0].shape[0], origin_logits[0].shape[1], attention_dim = None)
# origins_optimizer = optim.Adam(origins_alone_model.parameters(), lr=0.00001,  weight_decay=0.001)

In [ ]:
# train_one_style(origins_alone_model, origins_optimizer, R_origin_embeds, origin_logits, origin_token_ids, 100)

In [ ]:
def one_style_generate(prompt, tokenizer, SAT_model, GPT_transformer, context_to_sample, num_samples = 100, num_tokens_to_generate = 1, sen_to_generate = 10):
  #Put models in eval mode
    SAT_model.eval()
    GPT_transformer.eval()
    for sent in range(sen_to_generate):
        random_context_samples = random.sample(context_to_sample, num_samples)
        model_context_input =  torch.stack(random_context_samples, dim = 0)

        # tokenize prompt
        current_tokenization = tokenizer.encode(prompt, return_tensors = 'pt').to(device)

        for generation in range(num_tokens_to_generate):
            # put tokenized prompt through GPT_transformer to get GPT Logits
            GPT_logits = GPT_transformer(current_tokenization).last_hidden_state.squeeze()

            # put model_context_input and the GPT Logits into SAT model
            adjusted_output = SAT_model(model_context_input, GPT_logits)

            # Funtional softmax the SAT model output
            SM_adjusted_output = torch.nn.functional.softmax(adjusted_output, dim = 1)

            # argmax to get predicted token
            predicted_tokens = torch.argmax(SM_adjusted_output, dim =1)

            # get topk tokens
            top_predicted_tokens = torch.topk(SM_adjusted_output[-1], 2, dim =0).indices

            #dif way of getting top predicted
            predicted_token = top_predicted_tokens[0]
            if predicted_token == 247:
                print("WEIRD TOKEN PREDICTED")
                predicted_token = top_predicted_tokens[1]

            # print(torch.amax(SM_adjusted_output[-1]))

            # print(predicted_tokens[-1].unsqueeze(0).unsqueeze(0))
            # print(current_tokenization)

            current_tokenization = torch.cat((current_tokenization, predicted_token.unsqueeze(0).unsqueeze(0)), 1)
            # print(current_tokenization)

        # decode
        for i, beam in enumerate(current_tokenization):
            # print(f"{i}: {tokenizer.decode(beam)}")
            # print(f"{i}: {current_tokenization}")
            print(f"{i}: {tokenizer.decode(beam, skip_special_tokens=True)}")
    # for i, beam in enumerate(predicted_tokens):
    #     # if i == 0:
    #     #   continue
    #     print(f"{i}: {tokenizer.decode(beam, skip_special_tokens=True)} token_id: {beam}")


In [ ]:
# prompt = "In my younger and"

In [ ]:
# one_style_generate(prompt, Model_Import.tokenizer, gatsby_alone_model, Model_Import.head_transformer, R_gatsby_embeds, num_samples = 100, num_tokens_to_generate = 25)

In [ ]:
# one_style_generate(prompt, Model_Import.tokenizer, origins_alone_model, Model_Import.head_transformer, R_origin_embeds, num_samples = 100, num_tokens_to_generate = 25)

In [ ]:
# text_ids = Model_Import.tokenizer.encode(prompt, return_tensors = 'pt').to(device)
# base_generation = Model_Import.head_model.generate(text_ids, max_length=20)
# for i, beam in enumerate(base_generation):
#       print(f"{i}: {Model_Import.tokenizer.decode(beam, skip_special_tokens=True)}")

In [ ]:
import os

pos_path = "/home/ubuntu/IMDB_train/pos/"
neg_path = "/home/ubuntu/IMDB_train/neg/"

pos_token_list = []
pos_text_list = []
neg_token_list = []
neg_text_list = []

os.chdir(pos_path)
  

for file in os.listdir():
    with open(pos_path+file, 'r') as f:
        pos_text = f.read()
        # put into text list
        pos_text_list.append(pos_text)
        #tokenize and put into token list
        #pos_token_list.append(base_tokenizer.encode(pos_text, return_tensors = 'pt'))

os.chdir(neg_path)

for file in os.listdir():
    if not file.endswith('.txt'):
        continue
    with open(neg_path+file, 'r') as f:
        neg_text = f.read()
        # put into text list
        neg_text_list.append(neg_text)
        #tokenize and put into token list
        #neg_token_list.append(base_tokenizer.encode(neg_text_list, return_tensors = 'pt'))

In [ ]:
print(pos_text_list[20])
# test_text_pos = pos_text_list[0]

You know the story - a group of plucky no-hopers enter a competition they seemingly have no chance of winning - it's a tale that has been done to death by Hollywood (Bring It On, The Karate Kid, Escape to Victory, Best of the Best etc). Now Korea gives it a go with a Taekwondo team struggling for glory  and guess what  the result is predictable but ultimately satisfying.<br /><br />The fact that this movie doesn't fall flat on its face is down to the talented young cast who really make you care about the characters, and this in turn keeps you watching to the end.<br /><br />Fans of your typical martial arts movie may be disappointed  Taekwondo does not deliver the usual flurry of moves and acrobatics seen in most Kung Fu films; the action is limited to (albeit impressive) kicking and the occasional punch. This doesn't matter though, since it is the interaction of the characters and their fight to make something of themselves which makes this movie a success.


In [ ]:
from io import StringIO
from html.parser import HTMLParser

# ref https://stackoverflow.com/questions/753052/strip-html-from-strings-in-python
class MLStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.text = StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()
# cleaned = test_text_pos.replace('<br /><br />', ' ')
# print(cleaned)

def clean_imdb(review_list):
    for review in range(len(review_list)):
        cleaned_review = strip_tags(review_list[review])
        review_list[review] = cleaned_review

In [ ]:
clean_imdb(pos_text_list)
clean_imdb(neg_text_list)

In [ ]:
print(pos_text_list[0])

Legend of Dragoon is one of those little-known games that people either love or hate. Some people claim it's far too similar to other games, namely the Final Fantasy series--which is understandable, since it was originally intended to be Sony's equivalent of Final Fantasy. Honestly I can't comment on the similarities beyond that, as I'm not very familiar with the FF games.I think my favorite aspect of the game is the battle system. Not only do you have the ability to change into a more powerful dragoon form, but every time you attack, you have to pay attention in order to complete the attack by pressing buttons at the correct time. Not only that, sometimes enemies will attack you back right in the middle of a sequence, which means you have to press different buttons in order to avoid taking damage. Even the use of certain attack items requires a bit of button-mashing. If you don't want to attack, you can always guard, which not only cuts any damage taken in half, but raises your hit po

In [ ]:
len_sum = 0
count = 0
for sentence in pos_text_list:
    len_sum += len(sentence)
    count += 1
print(f"AVG LEN: {len_sum/count}")

AVG LEN: 1323.44904


In [ ]:
R_pos_embeds = make_context_embeddings_with_mean(Roberta_model, R_tokenizer, pos_text_list)

0
1000
2000
empty sentence
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000


In [ ]:
pos_logits, pos_token_ids = make_logits(Model_Import_6.head_transformer, Model_Import_6.tokenizer, pos_text_list)

0
1000
2000
empty sentence
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000


In [ ]:
print(len(pos_logits))
print(len(R_pos_embeds))

12499
12499


In [ ]:
R_neg_embeds = make_context_embeddings_with_mean(Roberta_model, R_tokenizer, neg_text_list)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000


In [ ]:
print(len(neg_text_list))

12500


In [ ]:
neg_logits, neg_token_ids = make_logits(Model_Import.head_transformer, Model_Import.tokenizer, neg_text_list)

In [ ]:
torch.save(R_pos_embeds, '/home/ubuntu/R_pos_embeds.pt')

In [ ]:
torch.save(pos_logits, '/home/ubuntu/pos_logits.pt')
torch.save(pos_token_ids, '/home/ubuntu/pos_token_ids.pt')

In [ ]:
torch.save(R_neg_embeds, '/home/ubuntu/R_neg_embeds.pt')

In [ ]:
torch.save(neg_logits, '/home/ubuntu/neg_logits.pt')
torch.save(neg_token_ids, '/home/ubuntu/neg_token_ids.pt')

In [ ]:
# x = torch.tensor([0, 1, 2, 3, 4])
# torch.save(x, 'tensor.pt')

In [ ]:
!pwd

/home/ubuntu/IMDB_train/neg


In [ ]:
import Model_Import_4

In [ ]:
import Model_Import_5

In [ ]:
pos_alone_model = Model_Import_5.DeeperModel(R_pos_embeds[0].shape[0], pos_logits[0].shape[1], attention_dim = None) # 1:42 per epoch on old deep model (2 layers)
pos_optimizer = optim.Adam(pos_alone_model.parameters(), lr=0.00001,  weight_decay=0.001)

In [ ]:
train_one_style(pos_alone_model, pos_optimizer, R_pos_embeds, pos_logits, pos_token_ids, 20)

In [ ]:
# neg_alone_model = Model_Import.ProposedModel(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], attention_dim = None) # 1:08
neg_alone_model = Model_Import_5.DeeperModel(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], attention_dim = None)
neg_optimizer = optim.Adam(neg_alone_model.parameters(), lr=0.00001,  weight_decay=0.001)

In [ ]:
train_one_style(neg_alone_model, neg_optimizer, R_neg_embeds, neg_logits, neg_token_ids, 20)

In [ ]:
import Model_Import_6

In [ ]:
# neg_alone_model = Model_Import_6.Test_skip_norm_model(R_neg_embeds[0].shape[0], neg_logits[0].shape[1], attention_dim = None)
# neg_optimizer = optim.Adam(neg_alone_model.parameters(), lr=0.00001,  weight_decay=0.001)

In [ ]:
prompt = "The movie"

In [ ]:
one_style_generate(prompt, Model_Import.tokenizer, pos_alone_model, Model_Import.head_transformer, R_pos_embeds, num_samples = 100, num_tokens_to_generate = 50)

In [ ]:
one_style_generate(prompt, Model_Import.tokenizer, neg_alone_model, Model_Import.head_transformer, R_neg_embeds, num_samples = 100, num_tokens_to_generate = 25)

In [ ]:
text_ids = Model_Import.tokenizer.encode(prompt, return_tensors = 'pt').to(device)
base_generation = Model_Import.head_model.generate(text_ids, max_length=20)
for i, beam in enumerate(base_generation):
      print(f"{i}: {Model_Import.tokenizer.decode(beam, skip_special_tokens=True)}")